In [2]:
from getpass import getpass
import os

OPENAI_API_KEY = getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)

from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

In [4]:
class CancerPatientProfile(BaseModel):
    patient_id: int
    patient_age: int
    patient_sex: str
    present_condition_history: str
    cancer_type: str
    cancer_stage: str
    cancer_location: str
    metastatic_locations: str
    past_medical_history: str
    symptoms: str
    allergies: str
    medications: str
    genetic_mutations: str
    diagnosis_staging: str
    imaging_reports: str
    pathological_findings: str
    molecular_testing: str
    treatment: str
    adjuvant_therapy: str
    metastatic_therapy: str
    medical_examinations: str
    follow_up_imaging: str
    treatment_outcomes: str
    family_history: str

In [5]:
examples = [
    {
        "example": """Patient ID: 1012, Age: 58, Sex: F,
History of Present Condition: Presented with persistent abdominal pain, unintentional weight loss, and changes in bowel habits. Colonoscopy revealed a 6 cm mass in the ascending colon.
Past Medical History: No significant medical history. Regular screenings revealed no abnormalities.
Symptoms: Persistent abdominal pain (right-sided), unintentional weight loss, changes in bowel habits (diarrhea and constipation).
Allergies: Allergic to Penicillin and Codeine.
Past and Present Medications: No regular medications.
Genetic Mutations: Underwent genetic testing revealing wild-type KRAS, NRAS, and BRAF.
Diagnosis Staging: T4aN1bM0, Stage IIIB colorectal adenocarcinoma.

Imaging Reports:
1. **Colonoscopy:**
   - Findings: 6 cm mass in the ascending colon, biopsy confirmed adenocarcinoma.

2. **CT Scan of Abdomen and Pelvis:**
   - Localized tumor in the ascending colon with involvement of adjacent structures.

3. **PET-CT Scan:**
   - Detected hypermetabolic activity in the primary tumor and regional lymph nodes.

Pathological Findings:
- Tumor Size: 6 cm
- Margin Status: Positive
- Lymphovascular Invasion: Present

Immunohistochemistry Results:
- CK20: Positive
- CDX2: Positive
- Mismatch Repair Proteins: Intact

Molecular Testing:
- KRAS Mutation: Wild-type
- NRAS Mutation: Wild-type
- BRAF Mutation: Wild-type

Treatment History:
- Underwent laparoscopic right hemicolectomy with clear surgical margins.

Adjuvant Chemotherapy:
- Initiated adjuvant chemotherapy with FOLFOX regimen.

Medical Examinations:
1. **Blood Tests:**
   - Normal complete blood count, liver and kidney function.

2. **Colonoscopic Biopsy Pathology:**
   - Histology: Moderately differentiated adenocarcinoma.
   - Tumor Grade: G2.

Follow-up Imaging:
- Regular CT scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Good response to adjuvant chemotherapy with no evidence of disease recurrence.

Family History: Limited family history; paternal grandfather with colorectal cancer at an advanced age.
"""
    },

    {
        "example": """Patient ID: 2022, Age: 45, Sex: F,
History of Present Condition: Noted a painless lump in the left breast during self-examination. Mammogram and biopsy confirmed invasive ductal carcinoma.
Past Medical History: No significant medical history. Regular mammograms were negative in previous years.
Symptoms: Painless lump in the left breast.
Allergies: Allergic to Codeine.
Past and Present Medications: Occasional use of over-the-counter pain relievers.
Genetic Mutations: Underwent genetic testing revealing BRCA mutations.
Diagnosis Staging: T2N0M0, Stage IIA invasive ductal carcinoma.

Imaging Reports:
1. **Mammogram:**
   - Detected a 2 cm irregular mass in the left breast.

2. **Breast Ultrasound:**
   - Confirmed the presence of the mass with no additional suspicious findings.

3. **Core Needle Biopsy:**
   - Histology: Invasive ductal carcinoma.

Pathological Findings:
- Tumor Size: 2 cm
- Hormone Receptor Status: ER-positive, PR-positive, HER2-negative.

Genomic Testing:
- Oncotype DX test performed, revealing a low recurrence score.

Surgical Procedures:
- Underwent lumpectomy with sentinel lymph node biopsy.

Adjuvant Therapy:
- Initiated adjuvant chemotherapy with AC-T regimen.
- Followed by adjuvant radiotherapy.

Endocrine Therapy:
- Prescribed tamoxifen for hormonal therapy.

Medical Examinations:
1. **Blood Tests:**
   - Normal complete blood count and liver function.
   - Monitoring tumor markers CA 15-3 and CEA.

2. **Cardiac Evaluation:**
   - Normal baseline cardiac function before starting chemotherapy.

Follow-up Imaging:
- Regular mammograms and breast ultrasounds scheduled for surveillance.

Treatment Outcomes:
- Completed adjuvant chemotherapy and radiotherapy with good tolerance.
- Ongoing hormonal therapy with tamoxifen.

Family History: Limited family history; no reported history of breast cancer or other malignancies.
"""
    },
    
    {
        "example": """Patient ID: 3030, Age: 60, Sex: M,
History of Present Condition: Presented with persistent changes in bowel habits, abdominal pain, and unintentional weight loss. Colonoscopy revealed a 4 cm mass in the rectum.
Past Medical History: History of irritable bowel syndrome (IBS) managed with dietary modifications. No prior history of colorectal issues.
Symptoms: Changes in bowel habits (diarrhea and constipation alternately), abdominal pain (lower left quadrant), unintentional weight loss.
Allergies: Allergic to Soy. Allergies to medications like Ibuprofen or Aspirin.
Past and Present Medications: Occasional use of over-the-counter antispasmodics for IBS symptoms.
Genetic Mutations: Underwent genetic testing revealing wild-type KRAS, NRAS, and BRAF.
Diagnosis Staging: T3N1M0, Stage III rectal adenocarcinoma.

Imaging Reports:
1. **Colonoscopy:**
   - Findings: 4 cm mass in the rectum, biopsy confirmed adenocarcinoma.

2. **CT Scan of Abdomen and Pelvis:**
   - Localized tumor in the rectum with regional lymph node involvement.

3. **MRI Pelvis:**
   - Detailed visualization of tumor extent and potential involvement of surrounding structures.

Pathological Findings:
- Tumor Size: 4 cm
- Differentiation: Moderately differentiated adenocarcinoma.

Molecular Testing:
- KRAS Mutation: Wild-type
- NRAS Mutation: Wild-type
- BRAF Mutation: Wild-type

Treatment:
- Underwent neoadjuvant chemoradiotherapy followed by low anterior resection.

Adjuvant Chemotherapy:
- Initiated adjuvant chemotherapy with FOLFOX regimen.

Medical Examinations:
1. **Blood Tests:**
   - Normal complete blood count and liver function.
   - Monitoring for tumor markers CEA and CA 19-9.

2. **Colonoscopic Surveillance:**
   - Scheduled for regular colonoscopic surveillance.

Follow-up Imaging:
- Regular CT scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Good response to neoadjuvant therapy and successful low anterior resection.
- Ongoing adjuvant chemotherapy with manageable side effects.

Family History: Limited family history; paternal uncle with colorectal polyps.
"""
    },
    

    {
        "example": """Patient ID: 4040, Age: 65, Sex: F,
History of Present Condition: Presented with persistent abdominal pain, jaundice, and unintended weight loss. Imaging studies revealed a 3 cm mass in the head of the pancreas.
Past Medical History: History of type 2 diabetes managed with oral medications. No prior history of pancreatic issues.
Symptoms: Abdominal pain (epigastric), jaundice, unintended weight loss.
Allergies: Allergic reactions or adverse events related to the use of heparin.
Past and Present Medications: Oral antidiabetic medication for type 2 diabetes.
Genetic Mutations: Underwent genetic testing revealing no known mutations.
Diagnosis Staging: T3N1M0, Stage II pancreatic adenocarcinoma.

Imaging Reports:
1. **Abdominal CT Scan:**
   - Findings: 3 cm mass in the head of the pancreas with involvement of the common bile duct.

2. **MRI Abdomen:**
   - Detailed visualization of tumor extent and potential vascular involvement.

3. **Endoscopic Ultrasound (EUS):**
   - Confirmed the presence of the pancreatic mass and obtained fine-needle biopsy for histology.

Pathological Findings:
- Tumor Size: 3 cm
- Differentiation: Poorly differentiated adenocarcinoma.

Molecular Testing:
- No actionable mutations identified.

Treatment:
- Underwent Whipple procedure for tumor resection.

Adjuvant Chemotherapy:
- Initiated adjuvant chemotherapy with gemcitabine and capecitabine.

Medical Examinations:
1. **Blood Tests:**
   - Monitoring liver function and complete blood count.
   - Regular monitoring of tumor markers CA 19-9 and CEA.

2. **Pancreatic Enzyme Replacement Therapy (PERT):**
   - Prescribed PERT due to post-surgical pancreatic insufficiency.

Follow-up Imaging:
- Regular CT scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Successful Whipple procedure with clear surgical margins.
- Managing chemotherapy-related side effects well.

Family History: Limited family history; no reported history of pancreatic cancer or related conditions.
"""
    },
    
   
    {
        "example": """Patient ID: 5050, Age: 55, Sex: M,
History of Present Condition: Presented with persistent cough, hemoptysis, and chest pain. Imaging studies revealed a 5 cm mass in the right upper lobe of the lung.
Past Medical History: Former smoker with a 20-pack-year history. No prior history of lung issues.
Symptoms: Persistent cough, hemoptysis, chest pain (right-sided).
Allergies: Allergic to contrast media.
Past and Present Medications: None reported.
Genetic Mutations: Underwent genetic testing revealing an EGFR mutation.
Diagnosis Staging: T2bN2M0, Stage IIIA non-small cell lung cancer (adenocarcinoma).

Imaging Reports:
1. **Chest X-ray:**
   - Initial finding of a suspicious mass in the right upper lobe.

2. **CT Scan of Chest:**
   - Confirmed a 5 cm mass in the right upper lobe with hilar and mediastinal lymph node involvement.

3. **Bronchoscopy with Biopsy:**
   - Histology: Adenocarcinoma with an EGFR mutation.

Molecular Testing:
- EGFR Mutation: Present.

Treatment:
- Underwent concurrent chemoradiotherapy with cisplatin and etoposide.

Surgical Consultation:
- Considered for surgical resection after initial chemoradiotherapy.

Medical Examinations:
1. **Pulmonary Function Tests:**
   - Evaluated lung function before and after treatment.

2. **PET-CT Scan:**
   - Post-treatment scan showed a partial response with decreased metabolic activity.

Follow-up Imaging:
- Regular CT scans and PET-CT scans for surveillance post-treatment.

Treatment Outcomes:
- Favorable response to concurrent chemoradiotherapy.
- Ongoing evaluation for potential surgical intervention.

Family History: Limited family history; no reported history of lung cancer or related conditions.
"""
    },
    

    {
        "example": """Patient ID: 6060, Age: 70, Sex: F,
History of Present Condition: Presented with persistent upper abdominal pain, unintentional weight loss, and difficulty swallowing. Endoscopy revealed a 6 cm mass in the gastric cardia.
Past Medical History: History of chronic gastritis managed with proton pump inhibitors. No prior history of gastric issues.
Symptoms: Persistent upper abdominal pain, unintentional weight loss, dysphagia.
Allergies: No known allergies.
Past and Present Medications: Proton pump inhibitors for chronic gastritis.
Genetic Mutations: Underwent genetic testing revealing no known mutations.
Diagnosis Staging: T3N1M0, Stage IIB gastric adenocarcinoma.

Imaging Reports:
1. **Upper Gastrointestinal Endoscopy:**
   - Findings: 6 cm mass in the gastric cardia.

2. **CT Scan of Abdomen:**
   - Localized tumor in the gastric cardia with regional lymph node involvement.

3. **Endoscopic Ultrasound (EUS):**
   - Detailed visualization of tumor depth and potential invasion.

Pathological Findings:
- Tumor Size: 6 cm
- Differentiation: Moderately differentiated adenocarcinoma.

Molecular Testing:
- No actionable mutations identified.

Treatment:
- Underwent total gastrectomy with D2 lymphadenectomy.

Adjuvant Chemotherapy:
- Initiated adjuvant chemotherapy with a fluoropyrimidine-based regimen.

Medical Examinations:
1. **Blood Tests:**
   - Monitoring complete blood count and liver function.
   - Regular follow-up for tumor markers CA 19-9 and CEA.

2. **Nutritional Assessment:**
   - Consulted with a dietitian for post-gastrectomy nutritional support.

Follow-up Imaging:
- Regular CT scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Successful total gastrectomy with clear surgical margins.
- Adjuvant chemotherapy ongoing with good tolerance.

Family History:
- Paternal grandmother diagnosed with gastric cancer at an advanced age.
- Maternal side has a history of colorectal polyps.
"""
    },
    
    {
        "example": """Patient ID: 8080, Age: 52, Sex: F,
History of Present Condition: Presented with persistent headaches, visual disturbances, and difficulty with memory. Brain imaging revealed a 5 cm mass in the left temporal lobe.
Past Medical History: History of hypertension managed with antihypertensive medication. No prior history of neurological issues.
Symptoms: Persistent headaches, visual disturbances, difficulty with memory.
Allergies: Allergic to Propofol and Isoflurane.
Past and Present Medications: Antihypertensive medication.
Genetic Mutations: Genetic testing revealed an IDH1 mutation and MGMT promoter methylation.
Diagnosis: Anaplastic astrocytoma, WHO Grade III.

Imaging Reports:
1. **MRI Brain:**
   - Findings: 5 cm mass in the left temporal lobe with surrounding edema.

2. **CT Scan of Head:**
   - Confirmed the presence of the brain mass.

3. **Stereotactic Biopsy:**
   - Histology: Anaplastic astrocytoma.

Molecular Testing:
- IDH1 Mutation: Present
- MGMT Promoter Methylation: Present

Treatment:
- Underwent stereotactic biopsy followed by concurrent chemoradiotherapy with temozolomide.

Adjuvant Chemotherapy:
- Initiated adjuvant temozolomide chemotherapy.

Medical Examinations:
1. **Neurological Assessments:**
   - Regular assessments for changes in cognitive function and motor skills.

2. **Blood Tests:**
   - Monitoring complete blood count and liver function.
   - Regular follow-up for general health status.

Follow-up Imaging:
- Regular MRI brain scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Partial resection achieved with stabilization of symptoms.
- Ongoing adjuvant chemotherapy with manageable side effects.

Family History: Limited family history; no reported history of brain cancer or related conditions.
"""
    },

    {
        "example": """Patient ID: 9090, Age: 48, Sex: F,
History of Present Condition: Presented with abdominal bloating, pelvic pain, and changes in bowel habits. Imaging studies revealed a 10 cm mass in the left ovary.
Past Medical History: History of polycystic ovary syndrome (PCOS). No prior history of gynecological issues.
Symptoms: Abdominal bloating, pelvic pain (left-sided), changes in bowel habits.
Allergies: Allergic to Heparin.
Past and Present Medications: Occasional use of over-the-counter pain relievers for PCOS-related discomfort.
Genetic Mutations: Underwent genetic testing revealing a BRCA1 mutation.
Diagnosis Staging: T3cN0M0, Stage IIIc high-grade serous ovarian carcinoma.

Imaging Reports:
1. **Pelvic Ultrasound:**
   - Findings: 10 cm mass in the left ovary with peritoneal involvement.

2. **CT Scan of Abdomen and Pelvis:**
   - Confirmed localized tumor in the left ovary with omental cake formation.

3. **CA-125 Blood Test:**
   - Elevated CA-125 levels.

Pathological Findings:
- Tumor Size: 10 cm
- Histology: High-grade serous ovarian carcinoma.

Molecular Testing:
- BRCA1 Mutation: Present

Treatment:
- Underwent total abdominal hysterectomy and bilateral salpingo-oophorectomy.

Adjuvant Chemotherapy:
- Initiated adjuvant chemotherapy with a carboplatin-paclitaxel regimen.

Medical Examinations:
1. **Blood Tests:**
   - Monitoring complete blood count and CA-125 levels.

2. **Genetic Counseling:**
   - Undertook genetic counseling for family members due to BRCA1 mutation.

Follow-up Imaging:
- Regular CT scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Successful surgical resection with no residual disease.
- Ongoing adjuvant chemotherapy with good tolerance.

Family History:
- Maternal side has a history of breast cancer; genetic testing pending for family members.
"""
    },
    
    {
        "example": """Patient ID: 9090, Age: 48, Sex: F,
History of Present Condition: Presented with abdominal bloating, pelvic pain, and changes in bowel habits. Imaging studies revealed a 10 cm mass in the left ovary.
Past Medical History: History of polycystic ovary syndrome (PCOS). No prior history of gynecological issues.
Symptoms: Abdominal bloating, pelvic pain (left-sided), changes in bowel habits.
Allergies: Allergic to Lisinopril and Metoprolol.
Past and Present Medications: Occasional use of over-the-counter pain relievers for PCOS-related discomfort.
Genetic Mutations: Underwent genetic testing revealing a BRCA1 mutation.
Diagnosis Staging: T3cN0M0, Stage IIIc high-grade serous ovarian carcinoma.

Imaging Reports:
1. **Pelvic Ultrasound:**
   - Findings: 10 cm mass in the left ovary with peritoneal involvement.

2. **CT Scan of Abdomen and Pelvis:**
   - Confirmed localized tumor in the left ovary with omental cake formation.

3. **CA-125 Blood Test:**
   - Elevated CA-125 levels.

Pathological Findings:
- Tumor Size: 10 cm
- Histology: High-grade serous ovarian carcinoma.

Molecular Testing:
- BRCA1 Mutation: Present

Treatment:
- Underwent total abdominal hysterectomy and bilateral salpingo-oophorectomy.

Adjuvant Chemotherapy:
- Initiated adjuvant chemotherapy with a carboplatin-paclitaxel regimen.

Medical Examinations:
1. **Blood Tests:**
   - Monitoring complete blood count and CA-125 levels.

2. **Genetic Counseling:**
   - Undertook genetic counseling for family members due to BRCA1 mutation.

Follow-up Imaging:
- Regular CT scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Successful surgical resection with no residual disease.
- Ongoing adjuvant chemotherapy with good tolerance.

Family History:
- Maternal side has a history of breast cancer; genetic testing pending for family members.
"""
    },

    {
        "example": """Patient ID: 11111, Age: 68, Sex: M,
History of Present Condition: Presented with urinary frequency, hesitancy, and nocturia. Digital rectal examination revealed an abnormal prostate. PSA levels were elevated.
Past Medical History: History of benign prostatic hyperplasia (BPH) managed with alpha-blockers. Also, diagnosed with hypertension and hyperlipidemia, managed with antihypertensive medication and statins. No prior history of prostate cancer.
Symptoms: Urinary frequency, hesitancy, nocturia.
Allergies: No known allergies.
Past and Present Medications: Alpha-blockers for BPH symptoms, antihypertensive medication, and statins.
Genetic Mutations: Underwent genetic testing revealing no known mutations.
Diagnosis Staging: T2bN0M0, Stage II prostate adenocarcinoma.

Imaging Reports:
1. **Transrectal Ultrasound (TRUS):**
   - Findings: 2 cm lesion in the right posterior lobe of the prostate.

2. **Prostate Biopsy:**
   - Histology: Adenocarcinoma with a Gleason score of 7 (3+4).

3. **MRI Prostate:**
   - Detailed visualization of tumor extent and potential extracapsular extension.

Pathological Findings:
- Tumor Size: 2 cm
- Gleason Score: 7 (3+4)

Molecular Testing:
- No actionable mutations identified.

Treatment:
- Underwent radical prostatectomy with pelvic lymph node dissection.

Adjuvant Therapy:
- Initiated adjuvant radiation therapy to the prostate bed.

Medical Examinations:
1. **PSA Monitoring:**
   - Regular monitoring of PSA levels post-prostatectomy.

2. **Uroflowmetry:**
   - Assessing postoperative urinary function.

Follow-up Imaging:
- Regular MRI scans for surveillance post-surgery and during adjuvant therapy.

Treatment Outcomes:
- Negative surgical margins with no evidence of residual disease.
- Ongoing adjuvant radiation therapy with well-tolerated side effects.

Family History:
- Paternal uncle diagnosed with prostate cancer at an older age.
"""
    },
    {
        "example": """Patient ID: 12121, Age: 55, Sex: F,
History of Present Condition: Noted a painless lump in the left breast during self-examination. Mammogram and biopsy confirmed invasive ductal carcinoma.
Past Medical History: Diagnosed with type 2 diabetes at the age of 45, managed with oral hypoglycemic agents. Also, history of hypertension diagnosed at 50, controlled with antihypertensive medication. Experienced a transient ischemic attack (TIA) at 53, currently on antiplatelet therapy. No prior history of breast issues.
Symptoms: Painless lump in the left breast.
Allergies: Allergic to Lidocaine.
Past and Present Medications: Oral hypoglycemic agents, antihypertensive medication, and antiplatelet therapy.
Genetic Mutations: Underwent genetic testing revealing no BRCA mutations.
Diagnosis Staging: T2N0M0, Stage IIA invasive ductal carcinoma.

Imaging Reports:
1. **Mammogram:**
   - Detected a 2 cm irregular mass in the left breast.

2. **Breast Ultrasound:**
   - Confirmed the presence of the mass with no additional suspicious findings.

3. **Core Needle Biopsy:**
   - Histology: Invasive ductal carcinoma.

Pathological Findings:
- Tumor Size: 2 cm
- Hormone Receptor Status: ER-positive, PR-positive, HER2-negative.

Genomic Testing:
- Oncotype DX test performed, revealing an intermediate recurrence score.

Surgical Procedures:
- Underwent lumpectomy with sentinel lymph node biopsy.

Adjuvant Therapy:
- Initiated adjuvant chemotherapy with AC-T regimen.
- Followed by adjuvant radiotherapy.

Endocrine Therapy:
- Prescribed aromatase inhibitor for hormonal therapy.

Medical Examinations:
1. **Blood Tests:**
   - Monitoring complete blood count, liver function, and HbA1c levels.

2. **Cardiac Evaluation:**
   - Regular cardiac assessments due to prior TIA.

Follow-up Imaging:
- Regular mammograms and breast ultrasounds scheduled for surveillance.

Treatment Outcomes:
- Successful lumpectomy with clear surgical margins.
- Completed adjuvant chemotherapy and radiotherapy with good tolerance.

Family History:
- Maternal side has a history of breast cancer; genetic counseling recommended for family members.
"""
    },
    
    {
        "example": """Patient ID: 13131, Age: 62, Sex: M,
History of Present Condition: Presented with persistent cough, hemoptysis, and chest pain. Imaging studies revealed a 5 cm mass in the right upper lobe of the lung.
Past Medical History: Diagnosed with chronic obstructive pulmonary disease (COPD) at the age of 55, managed with bronchodilators and inhaled corticosteroids. Also, history of coronary artery disease (CAD) with a previous myocardial infarction at 58, managed with antiplatelet therapy and statins. No prior history of lung issues.
Symptoms: Persistent cough, hemoptysis, chest pain (right-sided).
Allergies: Allergic to Ibuprofen and Aspirin.
Past and Present Medications: Bronchodilators, inhaled corticosteroids, antiplatelet therapy, and statins.
Genetic Mutations: Underwent genetic testing revealing no EGFR or ALK mutations.
Diagnosis Staging: T2bN2M0, Stage IIIA non-small cell lung cancer (adenocarcinoma).

Imaging Reports:
1. **Chest X-ray:**
   - Initial finding of a suspicious mass in the right upper lobe.

2. **CT Scan of Chest:**
   - Confirmed a 5 cm mass in the right upper lobe with hilar and mediastinal lymph node involvement.

3. **Bronchoscopy with Biopsy:**
   - Histology: Adenocarcinoma with no actionable mutations.

Molecular Testing:
- EGFR Mutation: Absent
- ALK Rearrangement: Absent

Treatment:
- Underwent concurrent chemoradiotherapy with cisplatin and etoposide.

Surgical Consultation:
- Considered for surgical resection after initial chemoradiotherapy.

Medical Examinations:
1. **Pulmonary Function Tests:**
   - Monitored lung function before and after treatment.

2. **Cardiac Evaluations:**
   - Regular assessments due to prior myocardial infarction.

Follow-up Imaging:
- Regular CT scans for surveillance post-treatment.

Treatment Outcomes:
- Favorable response to concurrent chemoradiotherapy.
- Ongoing evaluation for potential surgical intervention.

Family History:
- Paternal side has a history of COPD; genetic counseling recommended for family members.
"""
    },
    
    
{        "example": """Patient ID: 15151, Age: 55, Sex: M,
History of Present Condition: Presented with persistent changes in bowel habits, rectal bleeding, and abdominal pain. Colonoscopy revealed a 6 cm mass in the sigmoid colon.
Past Medical History: Diagnosed with hypertension at 50, managed with antihypertensive medication. No prior history of gastrointestinal issues.
Symptoms: Changes in bowel habits, rectal bleeding, abdominal pain (left lower quadrant).
Allergies: Allergic to Fentanyl and Morphine.
Past and Present Medications: Antihypertensive medication.
Genetic Mutations: Underwent genetic testing revealing no known mutations.
Diagnosis Staging: T3N1M0, Stage IIIA colorectal adenocarcinoma.

Imaging Reports:
1. **Colonoscopy:**
   - Detected a 6 cm mass in the sigmoid colon.

2. **CT Scan of Abdomen and Pelvis:**
   - Confirmed localized tumor with regional lymph node involvement.

3. **Biopsy:**
   - Histology: Moderately differentiated adenocarcinoma.

Molecular Testing:
- No actionable mutations identified.

Treatment:
- Underwent sigmoid colectomy with regional lymph node dissection.

Adjuvant Chemotherapy:
- Initiated adjuvant chemotherapy with FOLFOX regimen.

Medical Examinations:
1. **Colonoscopy Surveillance:**
   - Scheduled for regular follow-up colonoscopies.

2. **Blood Pressure Monitoring:**
   - Ongoing management of hypertension during cancer treatment.

Follow-up Imaging:
- Regular CT scans for surveillance post-surgery and during chemotherapy.

Treatment Outcomes:
- Successful sigmoid colectomy with clear surgical margins.
- Ongoing adjuvant chemotherapy with manageable side effects.

Family History:
- Maternal side has a history of colorectal polyps; genetic counseling recommended for family members.
"""
}
# Include other patient profiles in a similar format
]


In [ ]:
len(examples)

In [5]:
# Define the prompt template
SYNTHETIC_FEW_SHOT_PREFIX = """Given this {task}. You are a clinical assistant and you should generate a clinically rich and detailed synthetic cancer patient profile using the provided information as a foundation. 
You can go beyond the information provided in the examples to create a comprehensive cancer patient profile, but following the same logic as the exmaples provided to you.
Include extensive details to simulate a real medical case. Consider the patient's age, sex, present and past medical history, condition, current symptoms, diagnostic and therapeutic procedures,
imaging reports, genetic mutations, treatment modalities, treatment outcomes, and relevant family history. 
Ensure the profile is clinically plausible and reflects the complexities often encountered in oncology cases.  
Your goal is to create a thorough and informative narrative that healthcare professionals could encounter in a real-world clinical setting.\n\n"""

SYNTHETIC_FEW_SHOT_PREFIX = """Include comprehensive details to simulate a real medical case to complete the {subject}. See the provided context examples for inspiration, but you are free to generate any information not necessarily included in the examples. 
Consider the patient's age, sex, past medical history, current symptoms, diagnostic procedures, imaging reports, genetic mutations, treatment modalities, treatment outcomes, and relevant family history. Ensure the profile is plausible and reflects the complexities often encountered in oncology cases. 
Your goal is to create a thorough and informative narrative that healthcare professionals could encounter in a real-world clinical setting. Consider {extra} \n\n"""


OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["task", "extra"],
    example_prompt=OPENAI_TEMPLATE,
)


In [6]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=CancerPatientProfile,
    llm=ChatOpenAI(
        temperature=0.75,
    ),  # You'll need to replace with your actual Language Model instance
    prompt=prompt_template,
)

In [13]:
synthetic_results = []
for i in range(5):
    print(i)
    synthetic_results.extend(synthetic_data_generator.generate(
        subject="generation of synthetic cancer patient clinical data",
        extra="Patient ID should be 8 digits and generated randomly. Do not repeat the same patient more than once. Try to be creative in wording and details, do not sound monotonic",
        runs=20,
    )
    )

0
1
2
3
4


In [14]:
import json
for res in synthetic_results:
    
    file_path = "../data/synthetic_patients/%s.json"%res.patient_id

    # Save the dictionary to a JSON file
    with open(file_path, "w") as json_file:
        json.dump(res.dict(), json_file)

    print(f"Data saved to {file_path}")

Data saved to ../data/synthetic_patients/7070.json
Data saved to ../data/synthetic_patients/17171.json
Data saved to ../data/synthetic_patients/2020.json
Data saved to ../data/synthetic_patients/3030.json
Data saved to ../data/synthetic_patients/2525.json
Data saved to ../data/synthetic_patients/5050.json
Data saved to ../data/synthetic_patients/6060.json
Data saved to ../data/synthetic_patients/7071.json
Data saved to ../data/synthetic_patients/8080.json
Data saved to ../data/synthetic_patients/9999.json
Data saved to ../data/synthetic_patients/9090.json
Data saved to ../data/synthetic_patients/1111.json
Data saved to ../data/synthetic_patients/12345.json
Data saved to ../data/synthetic_patients/9998.json
Data saved to ../data/synthetic_patients/7777.json
Data saved to ../data/synthetic_patients/7778.json
Data saved to ../data/synthetic_patients/1010.json
Data saved to ../data/synthetic_patients/1357.json
Data saved to ../data/synthetic_patients/2468.json
Data saved to ../data/synthet

In [ ]:
import pickle
# Specify the file path where you want to save the list
file_path = "synthetic_list_data.pkl"

# Save the list to disk using pickle
with open(file_path, 'wb') as file:
    pickle.dump(synthetic_results, file)

print(f"The list has been saved to {file_path}")


In [ ]:
import json

# Specify the file path from which to load the list
file_path = "synthetic_list_data.json"

# Load the list from disk using json
with open(file_path, 'r') as file:
    loaded_list = json.load(file)

print(f"The loaded list is: {loaded_list}")

